In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

In [ ]:
# Connect to client libraries

openai = OpenAI()

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
grok_url = "https://api.x.ai/v1"

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)

In [ ]:
OPENAI_MODEL = "gpt-5-mini"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
GROK_MODEL = "grok-4"
GEMINI_MODEL = "gemini-2.5-pro"

In [ ]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

In [ ]:
message = f"""
Here is a report of the system information for my computer.
I want to run a Pytorch compiler to compile a single Pytorch file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any Pytorch compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile Pytorch code, then I'd like to run something like this in Python to compile and execute the code:

Please tell me exactly what I should use for the compile_command and run_command.

"""

response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))
    

In [ ]:
system_prompt = """
Your task is to convert Python code into high performance Pytorch
Respond only with Pytorch Do not provide any explanation other than occasional comments.
The Pytorch response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to Pytorch with the fastest possible implementation that produces identical output in the least time.

Respond only with Pytorch
Python code to port:

```python
{python}
```
"""

In [ ]:
def messages_for(python):
  return  [
    {"role":"system","content":system_prompt},
    {"role":"user","content":user_prompt_for(python)}
  ]



In [ ]:
def write_reply(reply):
  with open ("pytorch.py","w",encoding="utf-8") as f:
    f.write(reply)


In [ ]:
def port(client,model,python):
  response = client.chat.completions.create(model=model,messages=messages_for(python))
  reply = response.choices[0].message.content
  write_reply(reply)

In [ ]:
pi = """
import numpy as np
import time

# Data
x = np.random.rand(1_000_000)
w = np.random.rand(1_000_000)
b = 0.1

# Timing
start = time.perf_counter()

y = np.dot(x, w) + b

end = time.perf_counter()

print("NumPy Output:", y)
print("NumPy Time Taken:", end - start, "seconds")
"""

In [ ]:
def run_python(code):
 
    exec(code)

In [ ]:
run_python(pi)

In [ ]:
port(openai, OPENAI_MODEL, pi)

In [ ]:
port(anthropic, CLAUDE_MODEL, pi)
pt = """
import torch
import time

# Data size
n = 1_000_000

# Device (use GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Use double precision to match NumPy's default dtype
dtype = torch.float64

# Generate data on chosen device
x = torch.rand(n, dtype=dtype, device=device)
w = torch.rand(n, dtype=dtype, device=device)
b = 0.1

# Warm-up for CUDA (if used)
if device.type == "cuda":
    torch.cuda.synchronize()

# Timing
start = time.perf_counter()
y = torch.dot(x, w) + b
if device.type == "cuda":
    torch.cuda.synchronize()
end = time.perf_counter()

print("PyTorch Output:", y.item())
print("PyTorch Time Taken:", end - start, "seconds")
"""

In [ ]:
run_python(pt)

In [ ]:
run_python(pi)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.version.cuda)


In [ ]:
import os
import sys
import gradio as gr
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)
openai_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
ollama_url = "http://localhost:11434/v1"


In [ ]:
from openai import OpenAI

openai = OpenAI()

anthropic = OpenAI(api_key=anthropic_api_key,base_url=anthropic_url)
openai = OpenAI(api_key=openai_key)
gemini = OpenAI(api_key = google_api_key,base_url=gemini_url)
ollama = OpenAI(base_url=ollama_url,api_key="None")


In [ ]:
models= ["gpt-5","gemini-2.5-pro","llama"]
clients = {"gpt-5":openai,"gemini-2.5-pro":gemini,"llama":ollama}

In [ ]:
system_prompt = """
Your task is to convert Python code into high performance Pytorch
Respond only with Pytorch Do not provide any explanation other than occasional comments.
The Pytorch response needs to produce an identical output in the fastest possible time.
"""

def user_prompt_for(python):
    return f"""
Port this Python code to Pytorch with the fastest possible implementation that produces identical output in the least time.

Respond only with Pytorch
Python code to port:

```python
{python}
```
"""

In [ ]:
def messages_for(python):
  return [{"role":"user","content":user_prompt_for(python)},{"role":"system","content":system_prompt}]

In [ ]:
def write_output(python):
  with open("output_code.py","w") as f:
    f.write(python)


In [ ]:
def port(model,python):
  client = clients[model]
  res = client.chat.completions.create(model=model,messages = messages_for(python))
  reply = res.choices[0].message.content
  write_output(reply)
  return reply

In [ ]:
with gr.Blocks() as ui:
  with gr.Row():
    python = gr.Textbox(label="Python code: ",lines=28,value=pi)
    pytorch = gr.Textbox(label="Pytorch code: ",lines=28)
  with gr.Row():
    model = gr.Dropdown(models,label="Select model",value=models[0])
    convert=gr.Button("Convert Code")
  
  convert.click(port,inputs=[model,python],outputs = [pytorch])
ui.launch(inbrowser=True)

In [ ]:
import torch
import time

# Select fastest device available
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float64  # match NumPy default precision

# Data
x = torch.rand(1_000_000, dtype=dtype, device=device)
w = torch.rand(1_000_000, dtype=dtype, device=device)
b = torch.tensor(0.1, dtype=dtype, device=device)

# Timing
if device == "cuda":
    torch.cuda.synchronize()
start = time.perf_counter()

y = torch.dot(x, w) + b

if device == "cuda":
    torch.cuda.synchronize()
end = time.perf_counter()

print("PyTorch Output:", y.item())
print("PyTorch Time Taken:", end - start, "seconds")